# SPR 2026 - TF-IDF + SGDClassifier

**SGDClassifier:** Gradiente Estocástico - otimizado para dados esparsos

**Vantagens:**
- ✅ Treinamento incremental (stochastic gradient descent)
- ✅ Escala muito bem com número de features
- ✅ Suporta regularização L1/L2/ElasticNet
- ✅ Tempo de execução: **< 1 minuto**

---
**CONFIGURAÇÃO KAGGLE:**
1. Settings → Internet → **OFF**
2. **IMPORTANTE:** Execute "Run All" após commit
---

In [ ]:
# =============================================================================
# SPR 2026 - TFIDF + SGDCLASSIFIER (CÓDIGO CONSOLIDADO)
# =============================================================================
# Execute TODA esta célula - Tempo esperado: < 1 minuto
# =============================================================================

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - TF-IDF + SGDClassifier")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
np.random.seed(SEED)

# ==== CARREGAR DADOS ====
print("\n[1/4] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

# ==== TF-IDF ====
print("\n[2/4] Aplicando TF-IDF...")
tfidf = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train = tfidf.fit_transform(train['report'])
X_test = tfidf.transform(test['report'])
y_train = train['target'].values
print(f"    TF-IDF shape: {X_train.shape}")

# ==== SGDCLASSIFIER ====
print("\n[3/4] Treinando SGDClassifier...")
model = SGDClassifier(
    loss='log_loss',        # Log loss para classificação probabilística
    penalty='l2',
    alpha=1e-4,
    max_iter=1000,
    tol=1e-3,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1,
    early_stopping=True,
    validation_fraction=0.1
)
model.fit(X_train, y_train)
print("    ✓ Modelo treinado!")

# ==== SUBMISSÃO ====
print("\n[4/4] Gerando submissão...")
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})
submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ CONCLUÍDO - submission.csv criado!")
print("="*60)
print("\nDistribuição das predições:")
print(submission['target'].value_counts().sort_index())